In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_div = html.xpath('//div[@class="prod-fit-info"]/div[contains(@class, "mmy-row")]')
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for div in list_div:
                    make = div.xpath('./div[1]/text()')[0].strip()
                    list_model_year = div.xpath('./div[2]/div[@class="mmy-subrow"]/div[@class="mmy-subrow-title"]/text()')
                    for model_year in list_model_year:
                        list_vehicle.append(make + ' ' + model_year.strip())

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//section[@id="product-details"]/descendant::div[@class="prod-offer-item"]')
                
                # = = = = = = = = = = = = = = =
                
                list_td = html.xpath('//table[@class="spec-stripped-table"]/tr/td/text()')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Sku': '',
                                         'Title': '',
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': ''}])

                for div in list_div:
                    df_temp.loc[0, div.xpath('./div[@class="prod-offer-title"]/text()')[0].strip()[:-1]] = ' '.join(div.xpath('./div[@class="prod-offer-content"]/descendant-or-self::*/text()')).strip()

                start = ''
                for td in list_td:
                    if ':' in td:
                        if start == '':
                            df_temp.loc[0, td.split(':')[0]] = ':'.join(td.split(':')[1:])
                        else:
                            df_temp.loc[0, start + '_' + td.split(':')[0]] = ':'.join(td.split(':')[1:])
                    else:
                        start = td
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：204

https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-1555.html  <->  [ok] - 剩余数量：184 - 当前时间：16:26:32
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-91112.html  <->  [ok] - 剩余数量：183 - 当前时间：16:26:32
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-92145.html  <->  [ok] - 剩余数量：182 - 当前时间：16:26:32
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-96372.html  <->  [ok] - 剩余数量：181 - 当前时间：16:26:33
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-6700.html  <->  [ok] - 剩余数量：180 - 当前时间：16:26:33
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-97218.html  <->  [ok] - 剩余数量：179 - 当前时间：16:26:33
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-94731.html  <->  [ok] - 剩余数量：178 - 当前时间：16:26:34
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-91874.html  <->  [ok] - 剩余数量：177 - 当前时间：16:26:34
https://www.carid.com/ngk/laser-platinum-spark-plug.html  <->  [ok] - 剩余数量：176 - 当前时间：16:26:34
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-6701.html  

https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-5794.html  <->  [ok] - 剩余数量：106 - 当前时间：16:27:37
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-3106.html  <->  [ok] - 剩余数量：105 - 当前时间：16:27:40
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-5344.html  <->  [ok] - 剩余数量：104 - 当前时间：16:27:42
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-7418.html  <->  [ok] - 剩余数量：103 - 当前时间：16:27:44
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-7505.html  <->  [ok] - 剩余数量：102 - 当前时间：16:27:46
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-3508.html  <->  [ok] - 剩余数量：101 - 当前时间：16:27:47
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-3588.html  <->  [ok] - 剩余数量：100 - 当前时间：16:27:47
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-7866.html  <->  [ok] - 剩余数量：99 - 当前时间：16:27:50
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-5887.html  <->  [ok] - 剩余数量：98 - 当前时间：16:27:53
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-1402.html  <->  [ok]

https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-92085.html  <->  [ok] - 剩余数量：26 - 当前时间：16:30:16
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-1637.html  <->  [ok] - 剩余数量：25 - 当前时间：16:30:16
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-5476.html  <->  [ok] - 剩余数量：24 - 当前时间：16:30:18
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-91725.html  <->  [ok] - 剩余数量：23 - 当前时间：16:30:22
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-96393.html  <->  [ok] - 剩余数量：22 - 当前时间：16:30:26
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-95927.html  <->  [ok] - 剩余数量：21 - 当前时间：16:30:28
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-94524.html  <->  [ok] - 剩余数量：20 - 当前时间：16:30:28
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-6215.html  <->  [ok] - 剩余数量：19 - 当前时间：16:30:35
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-95853.html  <->  [ok] - 剩余数量：18 - 当前时间：16:30:36
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-90644.html  <->  [ok]